# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "city_data_csv.csv"
city_data = pd.read_csv(file)
city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bubaque,11.28,-15.83,78.15,70,1,15.73,GW,1585455663
1,coahuayana,18.73,-103.68,73.18,60,0,1.61,MX,1585455663
2,povenets,62.85,34.83,24.24,93,100,6.33,RU,1585455663
3,san quintin,30.48,-115.95,57.45,71,98,16.17,MX,1585455663
4,ushuaia,-54.80,-68.30,42.80,81,40,8.05,AR,1585455386
...,...,...,...,...,...,...,...,...,...
574,noumea,-22.28,166.46,80.01,65,90,18.34,NC,1585455655
575,ghanzi,-21.57,21.78,70.63,46,81,6.15,BW,1585455799
576,bulgan,48.81,103.53,38.66,33,88,4.36,MN,1585455799
577,rjukan,59.88,8.59,11.50,76,0,7.02,NO,1585455800


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key) 
locations = city_data[["Lat", "Lng"]].astype(float)
humidity = city_data['Humidity']

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_temp = city_data.loc[city_data['Max Temp']>70, :]
ideal_temp = ideal_temp.loc[city_data['Max Temp']<80, :]
ideal_temp = ideal_temp.loc[city_data['Wind Speed']<10, :]
ideal_temp = ideal_temp.loc[city_data['Cloudiness']==0, :]
ideal_temp = ideal_temp.reset_index(drop=True)
ideal_temp

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,coahuayana,18.73,-103.68,73.18,60,0,1.61,MX,1585455663
1,leogane,18.51,-72.63,77.04,71,0,1.77,HT,1585455667
2,inhambane,-23.86,35.38,72.12,73,0,7.43,MZ,1585455747
3,pokhara,28.23,83.98,73.24,18,0,2.13,NP,1585455699
4,floriano,-6.77,-43.02,74.43,82,0,2.71,BR,1585455772
5,san rafael,-34.62,-68.33,70.63,36,0,3.33,AR,1585455790
6,lazaro cardenas,17.96,-102.20,74.32,76,0,1.63,MX,1585455658


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
coordinates = ideal_temp[['Lat', 'Lng', 'City', 'Country']].copy()
coordinates

,Lat,Lng,City,Country
0,18.73,-103.68,coahuayana,MX
1,18.51,-72.63,leogane,HT
2,-23.86,35.38,inhambane,MZ
3,28.23,83.98,pokhara,NP
4,-6.77,-43.02,floriano,BR
5,-34.62,-68.33,san rafael,AR
6,17.96,-102.20,lazaro cardenas,MX


In [6]:
target_radius = 5000
target_type = "lodging"
    
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

for index, row in coordinates.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params['location']= f"{lat},{lng}"
    
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        response = requests.get(base_url, params=params)
        response = response.json()
        hotel = response['results'][0]['name']
        coordinates.loc[index,'Hotel'] = hotel
    except:
        print('sorry, skipping')
coordinates

,Lat,Lng,City,Country,Hotel
0,18.73,-103.68,coahuayana,MX,Hotel Los Arcos
1,18.51,-72.63,leogane,HT,Family Health Ministries Guest House
2,-23.86,35.38,inhambane,MZ,MHL Turismo SA
3,28.23,83.98,pokhara,NP,Hotel View Point
4,-6.77,-43.02,floriano,BR,Hotel Maktub
5,-34.62,-68.33,san rafael,AR,Hotel San Rafael
6,17.96,-102.20,lazaro cardenas,MX,Hotel Quinta Antigua


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in coordinates.iterrows()]
locations = coordinates[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig

# Display Map

Figure(layout=FigureLayout(height='420px'))